### Mechanism Y structured streaming code

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime
import pytz

spark.conf.set("spark.sql.shuffle.partitions", "4")

# ----------------------------
# Configuration
# ----------------------------
INPUT_PATH = "/Volumes/raw/staging/staging_volume/staging_data/"
OUTPUT_PATH = "/Volumes/processed/detection_data/processed_volume/output_detections_data/"
IMPORTANCE_TABLE = "googledrive.raw.customer_importance"
CHECKPOINT_PATH = "/Volumes/processed/detection_data/checkpoints/mechanism_y_streaming/"

# ----------------------------
# Load importance reference
# ----------------------------
imp_df = spark.read.table(IMPORTANCE_TABLE) \
    .select("source", "target", "type_trans", "weight") \
    .withColumn("source", trim(lower(regexp_replace(col("source"), "'", "")))) \
    .withColumn("target", trim(lower(regexp_replace(col("target"), "'", "")))) \
    .withColumn("type_trans", trim(lower(regexp_replace(col("type_trans"), "[^a-zA-Z0-9]", "")))) \
    .withColumn("weight", col("weight").cast("double")) \
    .dropna().cache()

# ----------------------------
# Utility
# ----------------------------
def ist_now():
    return datetime.now(pytz.timezone("Asia/Kolkata")).strftime('%Y-%m-%d %H:%M:%S')

# ----------------------------
# Pattern Detection Logic
# ----------------------------
def detect_patterns(df):
    ystart = ist_now()

    # Pattern 1: UPGRADE
    joined = df.join(imp_df,
        (df["merchant"] == imp_df["target"]) &
        (df["customer"] == imp_df["source"]) &
        (df["category"] == imp_df["type_trans"]),
        "inner"
    )

    agg = joined.groupBy("merchant", "customer") \
        .agg(count("*").alias("txn_count"),
             avg("weight").alias("avg_weight"))

    total_txns = joined.groupBy("merchant") \
        .agg(count("*").alias("total_txns"))

    p1_df = agg.join(total_txns, "merchant") \
        .filter((col("total_txns") >= 50000) &
                (col("txn_count") >= 169) &
                (col("avg_weight") <= 23)) \
        .select(
            lit(ystart).alias("YStartTime"),
            current_timestamp().alias("detectionTime"),
            lit("PatId1").alias("patternId"),
            lit("UPGRADE").alias("ActionType"),
            col("customer").alias("customerName"),
            col("merchant").alias("merchantId")
        )

    print(f"🔍 Pattern 1 - UPGRADE: {p1_df.count()} rows detected.")

    # Pattern 2: CHILD
    p2_df = df.groupBy("merchant", "customer") \
        .agg(avg("amount").alias("avg_amount"),
             count("*").alias("txn_count")) \
        .filter((col("avg_amount") < 23) &
                (col("txn_count") >= 8)) \
        .select(
            lit(ystart).alias("YStartTime"),
            current_timestamp().alias("detectionTime"),
            lit("PatId2").alias("patternId"),
            lit("CHILD").alias("ActionType"),
            col("customer").alias("customerName"),
            col("merchant").alias("merchantId")
        )

    print(f"🔍 Pattern 2 - CHILD: {p2_df.count()} rows detected.")

    # Pattern 3: DEI-NEEDED
    gender_df = df.withColumn("gender_cleaned", regexp_replace(col("gender"), "'", "")) \
                  .withColumn("gender_cleaned", trim(lower(col("gender_cleaned")))) \
                  .withColumn("gender_norm",
                      when(col("gender_cleaned").like("m%"), "Male")
                      .when(col("gender_cleaned").like("f%"), "Female")
                      .otherwise(None)
                  )

    p3_raw = gender_df.dropna(subset=["gender_norm", "merchant", "customer"]) \
        .select("merchant", "customer", "gender_norm") \
        .distinct() \
        .groupBy("merchant") \
        .pivot("gender_norm", ["Male", "Female"]) \
        .count().na.fill(0)

    p3_df = p3_raw.filter((col("Female") >= 100) & (col("Female") < col("Male"))) \
        .select(
            lit(ystart).alias("YStartTime"),
            current_timestamp().alias("detectionTime"),
            lit("PatId3").alias("patternId"),
            lit("DEI-NEEDED").alias("ActionType"),
            lit("").alias("customerName"),
            col("merchant").alias("merchantId")
        )

    print(f"🔍 Pattern 3 - DEI-NEEDED: {p3_df.count()} rows detected.")

    # Union all
    def empty_detection_df():
        return spark.createDataFrame([], schema="""
            YStartTime STRING,
            detectionTime TIMESTAMP,
            patternId STRING,
            ActionType STRING,
            customerName STRING,
            merchantId STRING
        """)

    final_df = (
        (p1_df if not p1_df.rdd.isEmpty() else empty_detection_df())
        .unionByName(p2_df if not p2_df.rdd.isEmpty() else empty_detection_df())
        .unionByName(p3_df if not p3_df.rdd.isEmpty() else empty_detection_df())
    )

    return final_df

# ----------------------------
# Streaming Ingestion
# ----------------------------
stream = spark.readStream.format("delta").load(INPUT_PATH)

# Clean the input
cleaned = stream \
    .withColumn("merchant", trim(lower(regexp_replace(col("merchant"), "[^a-zA-Z0-9]", "")))) \
    .withColumn("customer", trim(lower(regexp_replace(col("customer"), "[^a-zA-Z0-9]", "")))) \
    .withColumn("category", trim(lower(regexp_replace(col("category"), "[^a-zA-Z0-9]", "")))) \
    .withColumn("amount", col("amount").cast("double")) \
    .withColumn("gender", col("gender")) \
    .dropna(subset=["merchant", "customer"])

# ----------------------------
# foreachBatch Logic
# ----------------------------
def process_batch(micro_df, batch_id):
    if micro_df.isEmpty():
        print(f"🟡 Base batch {batch_id} contains no rows.")
        return

    micro_df.persist()
    detections = detect_patterns(micro_df)

    if not detections.rdd.isEmpty():
        detections = detections.withColumn("file_id", floor(monotonically_increasing_id() / 50))
        detections.write.format("delta").mode("append").partitionBy("file_id").save(OUTPUT_PATH)
        print(f"✅ Batch {batch_id}: {detections.count()} detections saved.")
    else:
        print(f"⚪ Batch {batch_id}: No pattern matches.")

    micro_df.unpersist()

# ----------------------------
# Start Streaming Query
# ----------------------------
query = cleaned.writeStream \
    .foreachBatch(process_batch) \
    .option("checkpointLocation", CHECKPOINT_PATH) \
    .trigger(processingTime="5 seconds") \
    .start()

query.awaitTermination()


### Mechanism Y code for Workflows and Jobs

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime
import pytz
import time

# ────── Spark Session ──────
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "4")

# ────── Configuration ──────
INPUT_PATH = "/Volumes/raw/staging/staging_volume/staging_data/"
OUTPUT_PATH = "/Volumes/processed/detection_data/processed_volume/output_detections_data/"
IMPORTANCE_TABLE = "googledrive.raw.customer_importance"
CHECKPOINT_PATH = "/Volumes/processed/detection_data/checkpoints/mechanism_y_streaming/"
FLAG_TABLE = "raw.staging.mechanism_flag"

# ────── Wait for Flag ──────
def is_flag_ready():
    try:
        return spark.sql(f"SELECT status FROM {FLAG_TABLE}").collect()[0]["status"] == "ready"
    except:
        return False

print("⏳ Waiting for Mechanism X to signal readiness via Delta flag table...")
while not is_flag_ready():
    print("🔁 Still waiting for flag = 'ready'...")
    time.sleep(10)
print("✅ Flag detected! Starting Mechanism Y...")

# ────── Load Importance Reference ──────
imp_df = spark.read.table(IMPORTANCE_TABLE) \
    .select("source", "target", "type_trans", "weight") \
    .withColumn("source", trim(lower(regexp_replace(col("source"), "'", "")))) \
    .withColumn("target", trim(lower(regexp_replace(col("target"), "'", "")))) \
    .withColumn("type_trans", trim(lower(regexp_replace(col("type_trans"), "[^a-zA-Z0-9]", "")))) \
    .withColumn("weight", col("weight").cast("double")) \
    .dropna().cache()

# ────── Time Helper ──────
def ist_now():
    return datetime.now(pytz.timezone("Asia/Kolkata")).strftime('%Y-%m-%d %H:%M:%S')

# ────── Pattern Detection ──────
def detect_patterns(df):
    ystart = ist_now()

    # Pattern 1: UPGRADE
    joined = df.join(imp_df,
        (df["merchant"] == imp_df["target"]) &
        (df["customer"] == imp_df["source"]) &
        (df["category"] == imp_df["type_trans"]),
        "inner"
    )

    agg = joined.groupBy("merchant", "customer") \
        .agg(count("*").alias("txn_count"),
             avg("weight").alias("avg_weight"))

    total_txns = joined.groupBy("merchant") \
        .agg(count("*").alias("total_txns"))

    p1_df = agg.join(total_txns, "merchant") \
        .filter((col("total_txns") >= 50000) &
                (col("txn_count") >= 169) &
                (col("avg_weight") <= 23)) \
        .select(
            lit(ystart).alias("YStartTime"),
            current_timestamp().alias("detectionTime"),
            lit("PatId1").alias("patternId"),
            lit("UPGRADE").alias("ActionType"),
            col("customer").alias("customerName"),
            col("merchant").alias("merchantId")
        )

    print(f"🔍 Pattern 1 - UPGRADE: {p1_df.count()} rows detected.")

    # Pattern 2: CHILD
    p2_df = df.groupBy("merchant", "customer") \
        .agg(avg("amount").alias("avg_amount"),
             count("*").alias("txn_count")) \
        .filter((col("avg_amount") < 23) &
                (col("txn_count") >= 8)) \
        .select(
            lit(ystart).alias("YStartTime"),
            current_timestamp().alias("detectionTime"),
            lit("PatId2").alias("patternId"),
            lit("CHILD").alias("ActionType"),
            col("customer").alias("customerName"),
            col("merchant").alias("merchantId")
        )

    print(f"🔍 Pattern 2 - CHILD: {p2_df.count()} rows detected.")

    # Pattern 3: DEI-NEEDED
    gender_df = df.withColumn("gender_cleaned", regexp_replace(col("gender"), "'", "")) \
                  .withColumn("gender_cleaned", trim(lower(col("gender_cleaned")))) \
                  .withColumn("gender_norm",
                      when(col("gender_cleaned").like("m%"), "Male")
                      .when(col("gender_cleaned").like("f%"), "Female")
                      .otherwise(None)
                  )

    p3_raw = gender_df.dropna(subset=["gender_norm", "merchant", "customer"]) \
        .select("merchant", "customer", "gender_norm") \
        .distinct() \
        .groupBy("merchant") \
        .pivot("gender_norm", ["Male", "Female"]) \
        .count().na.fill(0)

    p3_df = p3_raw.filter((col("Female") >= 100) & (col("Female") < col("Male"))) \
        .select(
            lit(ystart).alias("YStartTime"),
            current_timestamp().alias("detectionTime"),
            lit("PatId3").alias("patternId"),
            lit("DEI-NEEDED").alias("ActionType"),
            lit("").alias("customerName"),
            col("merchant").alias("merchantId")
        )

    print(f"🔍 Pattern 3 - DEI-NEEDED: {p3_df.count()} rows detected.")

    # Union all
    def empty_detection_df():
        return spark.createDataFrame([], schema="""
            YStartTime STRING,
            detectionTime TIMESTAMP,
            patternId STRING,
            ActionType STRING,
            customerName STRING,
            merchantId STRING
        """)

    final_df = (
        (p1_df if not p1_df.rdd.isEmpty() else empty_detection_df())
        .unionByName(p2_df if not p2_df.rdd.isEmpty() else empty_detection_df())
        .unionByName(p3_df if not p3_df.rdd.isEmpty() else empty_detection_df())
    )

    return final_df

# ────── Streaming Input ──────
stream = spark.readStream.format("delta").load(INPUT_PATH)

cleaned = stream \
    .withColumn("merchant", trim(lower(regexp_replace(col("merchant"), "[^a-zA-Z0-9]", "")))) \
    .withColumn("customer", trim(lower(regexp_replace(col("customer"), "[^a-zA-Z0-9]", "")))) \
    .withColumn("category", trim(lower(regexp_replace(col("category"), "[^a-zA-Z0-9]", "")))) \
    .withColumn("amount", col("amount").cast("double")) \
    .withColumn("gender", col("gender")) \
    .dropna(subset=["merchant", "customer"])

# ────── foreachBatch ──────
def process_batch(micro_df, batch_id):
    if micro_df.isEmpty():
        print(f"🟡 Batch {batch_id}: No new rows.")
        return

    micro_df.persist()
    detections = detect_patterns(micro_df)

    if not detections.rdd.isEmpty():
        detections = detections.withColumn("file_id", floor(monotonically_increasing_id() / 50))
        detections.write.format("delta").mode("append").partitionBy("file_id").save(OUTPUT_PATH)
        print(f"✅ Batch {batch_id}: {detections.count()} detections written.")
    else:
        print(f"⚪ Batch {batch_id}: No detections.")

    micro_df.unpersist()

# ────── Start Streaming Query ──────
query = cleaned.writeStream \
    .foreachBatch(process_batch) \
    .option("checkpointLocation", CHECKPOINT_PATH) \
    .trigger(processingTime="5 seconds") \
    .start()

query.awaitTermination()


### Verifying the writes

In [0]:
df = spark.read.format("delta").load("/Volumes/processed/detection_data/processed_volume/output_detections_data/")
df.display()

### Run this code before running the Mechanisms to get thresholds in hand

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, expr


# Load cleaned transaction data
transactions = spark.read.table("googledrive.raw.transactions") 

# Load cleaned importance reference data
imp_df = spark.read.table("googledrive.raw.customer_importance") 

# Join transaction and importance reference
joined = transactions.join(
    imp_df,
    (transactions["merchant"] == imp_df["target"]) &
    (transactions["customer"] == imp_df["source"]) &
    (transactions["category"] == imp_df["type_trans"]),
    "inner"
)

# Aggregate txn_count and avg_weight per (merchant, customer)
agg = joined.groupBy("merchant", "customer").agg(
    count("*").alias("txn_count"),
    avg("weight").alias("avg_weight")
)

# Total transactions per merchant
total_txns = joined.groupBy("merchant").agg(count("*").alias("total_txns"))

# Filter merchants with at least 50,000 total transactions
agg = agg.join(total_txns, "merchant").filter(col("total_txns") >= 50000)

# Precompute thresholds per merchant
thresholds_df = agg.groupBy("merchant").agg(
    expr("percentile_approx(txn_count, 0.9)").alias("txn_count_90"),
    expr("percentile_approx(avg_weight, 0.1)").alias("avg_weight_10")
)

thresholds_df.display()
# # Save the thresholds for real-time use in streaming pipeline
# thresholds_df.write.format("delta").mode("overwrite").save("/Volumes/processed/merchant_thresholds/")


### Pattern 1 - Debugging code

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, trim, lower, regexp_replace

# Load streaming batch or batch static data for debugging
df = spark.read.table("googledrive.raw.transactions")

# Load importance lookup once
imp_df = spark.read.table("googledrive.raw.customer_importance") \
    .select("source", "target", "type_trans", "weight") \
    .withColumn("source", trim(lower(regexp_replace(col("source"), "'", "")))) \
    .withColumn("target", trim(lower(regexp_replace(col("target"), "'", "")))) \
    .withColumn("type_trans", trim(lower(regexp_replace(col("type_trans"), "'", "")))) \
    .withColumn("type_trans", regexp_replace(col("type_trans"), "_", "")) \
    .withColumn("weight", col("weight").cast("double")) \
    .dropna() \
    .cache()

# Clean input columns for join keys
df_clean = df.withColumn("merchant", trim(lower(regexp_replace(col("merchant"), "[^a-zA-Z0-9]", "")))) \
             .withColumn("customer", trim(lower(regexp_replace(col("customer"), "[^a-zA-Z0-9]", "")))) \
             .withColumn("category", trim(lower(regexp_replace(col("category"), "[^a-zA-Z0-9]", "")))) \
             .dropna(subset=["merchant", "customer"])

# Join transactions with importance data
joined = df_clean.join(imp_df,
    (df_clean["merchant"] == imp_df["target"]) &
    (df_clean["customer"] == imp_df["source"]) &
    (df_clean["category"] == imp_df["type_trans"]),
    "inner"
)

# Aggregate metrics per (merchant, customer)
agg = joined.groupBy("merchant", "customer") \
            .agg(count("*").alias("txn_count"),
                 avg("weight").alias("avg_weight"))

# Total txn count per merchant
total_txns = joined.groupBy("merchant").agg(count("*").alias("total_txns"))

# Filter merchants with >= 50k transactions
agg_filtered = agg.join(total_txns, "merchant") \
                  .filter((col("total_txns") >= 50000) & (col("txn_count") >= 169) & (col("avg_weight") <= 23))

print(f"Pattern 1 detected rows: {agg_filtered.count()}")
agg_filtered.show(10, truncate=False)


### Pattern 2 - Debugging code

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, trim, lower, regexp_replace

# Load batch data for debugging
df = spark.read.table("googledrive.raw.transactions")

# Clean and cast columns
df_clean = df.withColumn("merchant", trim(lower(regexp_replace(col("merchant"), "[^a-zA-Z0-9]", "")))) \
             .withColumn("customer", trim(lower(regexp_replace(col("customer"), "[^a-zA-Z0-9]", "")))) \
             .withColumn("amount", col("amount").cast("double")) \
             .dropna(subset=["merchant", "customer", "amount"])

# Aggregate per (merchant, customer)
agg = df_clean.groupBy("merchant", "customer") \
    .agg(avg("amount").alias("avg_amount"),
         count("*").alias("txn_count"))

# Filter per pattern 2 criteria
p2_df = agg.filter((col("avg_amount") < 23) & (col("txn_count") >= 80))

print(f"Pattern 2 detected rows: {p2_df.count()}")
p2_df.show(10, truncate=False)


### Pattern 3 - Debugging code

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, trim, lower, regexp_replace

# Load batch data
df = spark.read.table("googledrive.raw.transactions")

# Clean columns: normalize strings, remove special chars, lowercase
df_clean = df.withColumn("merchant", trim(lower(regexp_replace(col("merchant"), "[^a-zA-Z0-9]", "")))) \
             .withColumn("customer", trim(lower(regexp_replace(col("customer"), "[^a-zA-Z0-9]", "")))) \
             .withColumn("gender", trim(lower(regexp_replace(col("gender"), "'", ""))))  # Remove single quotes

# Normalize gender after cleanup
gender_df = df_clean.withColumn("gender_norm",
    when(col("gender").like("m%"), "Male")
    .when(col("gender").like("f%"), "Female")
    .otherwise(None)
).dropna(subset=["merchant", "customer", "gender_norm"])

# Show cleaned gender normalization
gender_df.select("gender", "gender_norm").distinct().show()

# Distinct (merchant, customer, gender) to avoid duplicates
distinct_gender = gender_df.select("merchant", "customer", "gender_norm").distinct()

# Group by merchant and pivot on gender, count distinct customers
gender_counts = distinct_gender.groupBy("merchant") \
    .pivot("gender_norm", ["Male", "Female"]) \
    .count() \
    .na.fill(0)

# Filter merchants for Pattern 3
p3_df = gender_counts.filter((col("Female") >= 100) & (col("Female") < col("Male")))

print(f"Pattern 3 detected merchants: {p3_df.count()}")
p3_df.show(10, truncate=False)
